In [7]:
import unittest

from sqlalchemy import *
from sqlalchemy.orm import create_session
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime
from sqlalchemy.orm import contains_eager, joinedload
from sqlalchemy.orm import relationship

#Create and engine and get the metadata
Base = declarative_base()
engine = create_engine('sqlite:///Northwind_large.sqlite', echo=True)
metadata = MetaData(bind=engine)


#Reflect each database table we need to use, using metadata
class Customer(Base):
    __table__ = Table('Customer', metadata, autoload=True)
    orders = relationship("Order", backref="customer")

class Shipper(Base):
    __table__ = Table('Shipper', metadata, autoload=True)
    orders = relationship("Order", backref="shipper")

class Employee(Base):
    __table__ = Table('Employee', metadata, autoload=True)
#    orders = relationship("Order", backref="employee")
    territories = relationship('Territory', secondary=Table('EmployeeTerritory', metadata, autoload=True))

class Territory(Base):
    __table__ = Table('Territory', metadata, autoload=True)
    region = relationship('Region', backref='territories')

class Region(Base):
    __table__ = Table('Region', metadata, autoload=True)


class Order(Base):
    __table__ = Table('Order', metadata, autoload=True)
    products = relationship('Product', secondary=Table('OrderDetail', metadata, autoload=True))
    employee = relationship('Employee', backref='orders')

class Product(Base):
    __table__ = Table('Product', metadata, autoload=True)
    supplier = relationship('Supplier', backref='products')
    category = relationship('Category', backref='products') 

class Supplier(Base):
    __table__ = Table('Supplier', metadata, autoload=True)

class Category(Base):
    __table__ = Table('Category', metadata, autoload=True)


class Test(unittest.TestCase):

    def setUp(self):
        #Create a session to use the tables    
        self.session = create_session(bind=engine)        

    def tearDown(self):
        self.session.close()

    def test_withJoins(self):
        q = self.session.query(Customer)
        q = q.join(Order)
        q = q.join(Shipper)
        q = q.filter(Customer.CustomerID =='ALFKI')
        q = q.filter(Order.OrderID=='10643')
        q = q.filter(Shipper.ShipperID=='1')
        q = q.options(contains_eager(Customer.orders, Order.shipper))
        res = q.all()
        cus = res[0]
        ord = cus.orders[0]
        shi = ord.shipper
        self.assertEqual(shi.Phone, '(503) 555-9831')

2020-02-02 13:48:11,658 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-02 13:48:11,663 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,667 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-02 13:48:11,670 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,676 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Customer")
2020-02-02 13:48:11,679 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,686 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Customer' AND type = 'table'
2020-02-02 13:48:11,689 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,692 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("Customer")
2020-02-02 13:48:11,694 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,695 INFO sqlalchemy.engine.base.Engine PRAGM

2020-02-02 13:48:11,945 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("Territory")
2020-02-02 13:48:11,949 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,952 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Territory' AND type = 'table'
2020-02-02 13:48:11,955 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,957 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_list("Territory")
2020-02-02 13:48:11,960 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,962 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_list("Territory")
2020-02-02 13:48:11,963 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,965 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_info("sqlite_autoindex_Territory_1")
2020-02-02 13:48:11,966 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:11,968 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM

2020-02-02 13:48:12,171 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Product' AND type = 'table'
2020-02-02 13:48:12,172 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:12,183 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Supplier")
2020-02-02 13:48:12,184 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:12,191 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Supplier' AND type = 'table'
2020-02-02 13:48:12,193 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:12,201 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("Supplier")
2020-02-02 13:48:12,204 INFO sqlalchemy.engine.base.Engine ()
2020-02-02 13:48:12,207 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("Supplier")
2020-02-02 13:48:12,210 INFO sqlalchemy.engine.base.Engine ()
2020-02-0